<a href="https://colab.research.google.com/github/gabriellaec/iris_recognition/blob/main/Visa%CC%83o_Computacional_Aula_04_(1)_(5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto 1**: Parte um






## Bibliotecas importadas

In [1]:
import numpy as np
import cv2 as cv
import cv2
# from google.colab.patches import cv2_imshow
import sys
from matplotlib import pyplot as plt
%matplotlib inline
import os
import G6_iris_recognition
import random
from skimage import measure

## Funções auxiliares

In [2]:
# Função para mostrar várias imagens em forma de uma grade

def image_grid(images, cols = 2):
    n_images = len(images)
    fig = plt.figure()
    for n, (image) in enumerate(images):
        a = fig.add_subplot(cols, np.ceil(n_images/float(cols)), n + 1)
        if image.ndim == 2:
            plt.gray()
        plt.imshow(image)
    fig.set_size_inches(np.array(fig.get_size_inches()) * n_images)
    plt.show()

In [3]:
# Filtro de sobel

def sobel(img_gray):
    sobelx = cv.Sobel(img_gray,cv.CV_64F,1,0)
    sobely = cv.Sobel(img_gray,cv.CV_64F,0,1)
    return sobelx

In [4]:
# Função para isolar o maior contorno de uma imagem

def isolate_eye(img_gray):
      T, thresh  = cv.threshold(img_gray,0,255,cv.THRESH_BINARY+cv.THRESH_OTSU)
      img_contours, _ = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
      cv.drawContours(img_gray, img_contours, -1, (0, 255, 0))
      img_contours = sorted(img_contours, key=cv.contourArea)
      for i in img_contours:
        if cv.contourArea(i) > 1100:
            break
      mask = np.zeros(img_gray.shape[:2], np.uint8)
      cv.drawContours(mask, [i],-1, 255, -1)
      new_img = cv.bitwise_and(img_gray, img_gray, mask=mask)
      eye = img_gray-new_img
      return eye

In [5]:
# Função para detectar contornos (Canny)

def auto_canny(image, sigma=0.33):
    v = np.median(image)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv.Canny(image, lower, upper)
    return edged

In [6]:
# Função para isolar o olho e aplicar um filtro bilateral

def blurred_eyes(img_gray):
    outimg = cv.threshold(cv.medianBlur(img_gray, 3), 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)[1]
    img_gray[outimg==255]=255
    processed_img = cv.bilateralFilter(img_gray,9,75,75)
    return processed_img

In [7]:
# Função para aplicar um equalizador de histograma adaptativo (CLAHE)
# E também faz um ROI para isolar o olho da imagem

def adaptive_histeq(img_gray):
      clahe = cv.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
      final_img = clahe.apply(img_gray) 

      edges = auto_canny(img_gray) 
      indices = np.where(edges != [0])

      # cria uma nova imagem em branco
      blank_img = np.zeros([480, 640],dtype=np.uint8)
      blank_img[:] = 255  

      h_max = max(indices[0])
      h_min = min(indices[0])

      w_max = max(indices[1])
      w_min = min(indices[1])

      roi=final_img[h_min:h_max,w_min:w_max]
      blank_img[h_min:h_max,w_min:w_max]=roi
      return blank_img

In [8]:
# Função de faz um ROI com base nos contornos da imagem
# Usada para isolar o olho

def edges_roi(img_gray):
      edges = auto_canny(img_gray) 
      indices = np.where(edges != [0])

      # cria uma nova imagem em branco com as dimensões padrão
      blank_img = np.zeros([480, 640],dtype=np.uint8)
      blank_img[:] = 255  

      h_max = max(indices[0])
      h_min = min(indices[0])

      w_max = max(indices[1])
      w_min = min(indices[1])

      roi=img_gray[h_min:h_max,w_min:w_max]
      blank_img[h_min:h_max,w_min:w_max]=roi
      return blank_img

In [9]:
# Função para aplicar um equalizador de histograma adaptativo (CLAHE)

def apply_clahe(img_gray):
      clahe = cv.createCLAHE(clipLimit = 2.0, tileGridSize=(8,8))
      final_img = clahe.apply(img_gray) 
      return final_img

In [10]:
# Função para aplicar um equalizador de histograma adaptativo (CLAHE)

def apply_intense_clahe(img_gray):
      clahe = cv.createCLAHE(clipLimit = 5.0, tileGridSize=(12,12))
      final_img = clahe.apply(img_gray) 
      return final_img

In [11]:
# Função que aplica uma equalização do histograma no canal V da imagem

def hsv_histeq(img_gray):
    value=90
    bgr = cv.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
    hsv = cv.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    imgeq=cv.equalizeHist(v)

    final_hsv = cv2.merge((h, s, imgeq))
    img = cv.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    img_gray = cv.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img_gray

In [12]:
# Função que tenta melhorar a equalização do histograma ao aplicar 
# um borramento na parte externa e destacar o olho

def improved_histeq(img_gray):
      imgeq=hsv_histeq(img_gray)
      equalized_eye_original = isolate_eye(imgeq.astype(np.uint8))
      equalized_eye=apply_clahe(equalized_eye_original)
      blurc=cv.GaussianBlur(img_gray,(11,11),0)
      
      rows, cols = img_gray.shape
      processed_img=np.zeros([rows,cols],dtype=np.uint8)
      for l in range (rows):
          for c in range (cols):
            if equalized_eye_original[l,c] == 0:
                processed_img[l,c] = blurc[l,c]
            else:
                processed_img[l,c] = (equalized_eye[l,c])
      return processed_img

In [13]:
# Função que usa uma máscara de binarização para remover o reflexo no olho

# adaptado do tutorial: https://rcvaram.medium.com/glare-removal-with-inpainting-opencv-python-95355aa2aa52

def create_mask(gray):
    blurred = cv2.GaussianBlur( gray, (9,9), 0 )
    _,thresh_img = cv2.threshold( blurred, 180, 255, cv2.THRESH_BINARY)
    thresh_img = cv2.erode( thresh_img, None, iterations=2 )
    thresh_img  = cv2.dilate( thresh_img, None, iterations=4 )
    return thresh_img

def remove_glare(img_gray):
    blurred = cv2.GaussianBlur(img_gray, (9,9), 0)
    mask=create_mask(blurred)
    _,thresh_img = cv2.threshold( blurred, 180, 255, cv2.THRESH_BINARY)
    dst = cv.inpaint(img_gray,mask,30,cv2.INPAINT_TELEA)

    return dst

In [14]:
# Função que ajusta o canal V da imagem HSV para aumentar o brilho da imagem

# Adaptada da fonte: https://www.ti-enxame.com/pt/python/como-mudar-rapidamente-o-brilho-da-imagem-com-python-opencv/1055807330/
def increase_brightness(img):
    value=80
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

## Processamento de imagens

Na etapa a seguir, foram testadas diferentes técnicas de processamento de imagens, com o intuito de avaliar e selecionar aquelas com os melhores efeitos para deixar as íris em evidência ou minimizar os detalhes das partes externas que atrapalham o reconhecimento.

Esta etapa foi dividida em filtros cuja função consiste em aumentar ou em diminuir os detalhes. Em seguida, foram aplicadas técnicas para melhor isolar a íris. Por fim, os melhores resultados foram selecionados para serem aplicados no modelo no próximo notebook (Parte Dois)

In [ ]:
original_imgs_list=[]

# iteração 0: equalização do histograma 
equalized_imgs_list=[]

#iteração 1: blur 1 (mais suave)
blurred_imgs_list=[]

# iteração 2: blur 2 (mais forte)
blurred2_imgs_list=[]

#iteração 7: blur muito intenso
blurred3_imgs_list=[]

#iteração 3: sobel
sobel_list=[]

# iteração 4: laplacian
laplacian_list=[]

#iteração 5: contornos
edges_imgs_list=[]

# iteração 6: isolando o olho
eye_imgs_list=[]

# iteração 8: recortando o olho da imagem aplicando uma máscara de threshold + filtro bilateral
eyes2_list=[]
matches_eyes2=0

# iteração 9: CLAHE
CLAHE_list=[]

intense_CLAHE_list=[]

convolution_product_list=[]

improved_hist_list=[]

hsv_histeq_list=[]

denoised_list=[]

detailEnhance_list=[]

for iteration in range(15):
    for dir in range(0,60):
      for file in range(0,2):

          src_path=f"images2/00{dir:02d}/00{dir:02d}_0{file:02d}.bmp"
          # processamento da imagem
          if os.path.exists(src_path):
              img_gray = cv.imread(src_path, cv.IMREAD_GRAYSCALE) 
              imgeq=cv.equalizeHist(img_gray)

              if iteration == 0:
                  imgeq=cv.equalizeHist(img_gray)
                  equalized_imgs_list.append(imgeq)
                  original_imgs_list.append(img_gray)
                    
              elif iteration == 1:
                  img_processed=cv.GaussianBlur(img_gray,(7,7),0) 
                  blurred_imgs_list.append(img_processed)
                
              elif iteration == 2:
                  img_processed=cv.medianBlur(img_gray, 5)  
                  blurred2_imgs_list.append(img_processed)

              elif iteration == 3:
                  img_processed=sobel(img_gray)
                  sobel_list.append(img_processed.astype(np.uint8))
              elif iteration == 4:
                  img_processed = cv.Laplacian(imgeq,cv.CV_64F)
#                   img_processed=img_gray+lap
                  laplacian_list.append(img_processed.astype(np.uint8))
                    
              elif iteration == 5:
                  img_processed=imgeq
                  edges = auto_canny(imgeq)
                  img_processed[edges==255] = (cv.medianBlur(img_gray, 17))[edges==255]
                  edges_imgs_list.append(img_processed)
                
              elif iteration == 6:
                   img_processed = isolate_eye(img_gray)
                   eye_imgs_list.append(img_processed)
                
              elif iteration == 7:
                   img_processed=blurred_eyes(img_gray)
                   eyes2_list.append(img_processed)
                
              elif iteration == 8:
                   img_processed=adaptive_histeq(img_gray)
                   CLAHE_list.append(img_processed)
                
              elif iteration == 9:
                   img_processed=improved_histeq(img_gray)
                   img_processed=remove_glare(img_processed)
                   improved_hist_list.append(img_processed)
                
              elif iteration == 10:
                   img_processed=hsv_histeq(img_gray)
                   img_processed=edges_roi(img_processed)
                   hsv_histeq_list.append(img_processed)
                    
              elif iteration == 11:
                   img=cv2.cvtColor(img_gray, cv2.COLOR_GRAY2BGR)
                   img_processed= cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
                   img_processed = cv2.cvtColor(img_processed, cv2.COLOR_BGR2GRAY)
                   denoised_list.append(img_processed)
              elif iteration == 12:
                   img_processed=cv.GaussianBlur(img_gray,(3,3),0) 
                   img_processed=apply_intense_clahe(img_processed)
                   intense_CLAHE_list.append(img_processed)
              elif iteration == 13:
                   kernel = np.array([[0, -1, 0],
                                       [-1, 7,-1],
                                      [0, -1, 0]])
                   image_sharp = cv2.filter2D(src=imgeq, ddepth=-1, kernel=kernel)
                   convolution_product_list.append(image_sharp)
              elif iteration == 14:
                   cimg = cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR)
                   img_processed = cv.detailEnhance(cimg)
                   img_gray = cv2.cvtColor(img_processed,cv2.COLOR_BGR2GRAY)             
                   detailEnhance_list.append(img_gray)
              

            

## Exemplos das imagens originais

In [ ]:
random_imgs = random.sample(original_imgs_list, 6)
result = image_grid(np.array(random_imgs))

## 1. Aumentando os detalhes das imagens

Os filtros a seguir são muito úteis para as imagens em que as especificidades da íris são originalmente mais difíceis de identificar.

## Equalização do histograma

Quando as cores da imagem estão muito próximas, é mais difícil perceber a divisão entre elas.

A solução para isso é a equalização do histograma de cores, que é utilizada para melhor distribuir as cores da imagem e assim aumentar o contraste. Isso seria bom para o algoritmo conseguir perceber os contornos e as características da íris mais facilmente.

In [ ]:
random_imgs = random.sample(equalized_imgs_list, 6)
result = image_grid(np.array(random_imgs))

Como podemos ver, agora os detalhes de cada íris são mais facilmente identificados. Além disso, fica mais fácil agora perceber os limites de onde a íris se encontra. Porém, a imagem parece ter ficado muito escura e com muito reflexo na pele que circunda o olho.

## Equalização de histograma do canal V


Outro procedimento explorado foi a equalização do histograma aplicada em uma imagem HSV somente no canal V. Este canal é aquele responsável pelo brilho.

In [ ]:
random_imgs = random.sample(hsv_histeq_list, 6)
result = image_grid(np.array(random_imgs))

As imagens mostram que os resultados da equalização do histograma no canal V foram bastante semelhantes aos da primeira equalização aplicada, porém neste procedimento, a íris ficou mais clara.

## Equalização do histograma adaptativa (CLAHE) mais suave

Com a equalização adaptativa do histograma, a imagem é dividida em uma grade M x N. Em seguida, é aplicada a equalização a cada célula da grade, resultando em uma imagem de saída de maior qualidade. Com isso, esse filtro consegue melhorar o contraste da imagem sem aumentar o ruído. 

In [ ]:
random_imgs = random.sample(CLAHE_list, 6)
result = image_grid(np.array(random_imgs))

Este filtro apresentou um desempenho muito bom, na medida que intensificou muito os detalhes da íris, sem destacar excessivamente as partes externas ao olho.

## Equalização do histograma adaptativa (CLAHE) mais intensa

A função do CLAHE OpenCv possui 2 parâmetros:
* **clipLimit:** Threshold para limitação de contraste
* **tileGridSize:** Divide a imagem de entrada em blocos M x N e, em seguida, aplica a equalização do histograma a cada bloco local

Nas imagens apresentadas a seguir, foi aumentado o parâmetro clipLimit, visando a aumentar ainda mais o contraste.

In [ ]:
random_imgs = random.sample(intense_CLAHE_list, 6)
result = image_grid(np.array(random_imgs))

Percebe-se que com a mudança dos coeficientes da função, os detalhes da íris ficaram mais evidentes. No entanto, os cílios também ficaram com maior destaque, o que pode atrapalhar a identificação pelo modelo.

#### Tentativa de melhorar o resultado

Para melhorar os resultados da equalização de histograma apresentada anteriormente, um procedimento de identificação do maior contorno foi aplicado, com o intuito de isolar o olho do restante da imagem. Após separada, a área de interesse foi clareada, por meio do ajuste do brilho da imagem. Nas áreas externas ao olho, foi aplicado um filtro de borramento, cujo intuito era diminuir os detalhes. Os resultados estão exibidos a seguir:

In [ ]:
random_imgs = random.sample(improved_hist_list, 6)
result = image_grid(np.array(random_imgs))

Percebe-se que o resultado não ficou tão bom como esperado, principalmente por conta de algumas falhas na máscara, que ao identificar o maior contorno, acaba por não isolar o olho completamente como seria ideal. Dessa forma, a imagem fica com algumas "manchas", que atrapalham a identificação.

## Filtro de Sobel

O operador Sobel detecta bordas marcadas por mudanças repentinas na intensidade do pixel, sendo muito útil na identificação de contornos.

In [ ]:
random_imgs = random.sample(sobel_list, 6)
result = image_grid(np.array(random_imgs))

As imagens não ficaram boas para a identificação após a aplicação deste filtro, com muito ruído e pouco destaque às íris.

## Detail Enhance

A função detailEnhance() da OpenCv é muito útil para aumentar os detalhes de uma imagem.

In [ ]:
random_imgs = random.sample(detailEnhance_list, 6)
result = image_grid(np.array(random_imgs))

O filtro parece de fato ter aumentado a quantidade de detalhes na imagem, podendo assim ser útil para a identificação da íris.

## Filtro de convolução

Nesta etapa, foi usada a função filter2D da OpenCv, que aplica uma matriz de convolução na imagem, visando a aumentar o contraste de um pixel com seus vizinhos.

In [ ]:
random_imgs = random.sample(convolution_product_list, 6)
result = image_grid(np.array(random_imgs))

Os resultados parecem ter ficados muito bons, já que os detahes da íris ficaram bem evidenciados e o reflexo da pele foi removido. Contudo, os cílios também parecem ter ficado muito em evidência.

## Filtro Laplaciano

Este filtro ultiliza uma derivada segunda para detectar os contornos da imagem.

In [ ]:
random_imgs = random.sample(laplacian_list, 6)
result = image_grid(np.array(random_imgs))

As imagens resultantes parecem não ser adequadas para o treinamento do modelo, já que há muito ruído e as íris não ficaram em destaque.

# 2. Diminuindo os detalhes

Diminuir os detalhes é muito útil para os casos de imagens que contém elementos atrapalhando a identificação das íris, como por exemplo cílios, sombrancelhas e reflexos na pele.

## Borramento mais leve

Primeiramente, foi aplicado um borramento mais leve, um filtro Gaussian Blur com uma matriz 7x7.

In [ ]:
random_imgs = random.sample(blurred_imgs_list, 6)
result = image_grid(np.array(random_imgs))

Como podemos ver, o borramento foi bastante suave e permite que as peculiaridades da íris ainda sejam identificadas.

## Borramento mais intenso

Para retirar ainda mais detalhes que atrapalham o reconhecimento, foi aplicado um filtro mais intenso para o borramento: o Median Blur

In [ ]:
random_imgs = random.sample(blurred2_imgs_list, 6)
result = image_grid(np.array(random_imgs))

O procedimento acima acabou tirando detalhes demais da íris, o que infuenciaria negativamente a identificação pelo modelo

## Remoção de ruídos

Este filtro tem a função de remover os ruídos da imagem e foi aplicado com o intuito de diminuir o efeito dos cílios na identificação do modelo.

In [ ]:
random_imgs = random.sample(denoised_list, 6)
result = image_grid(np.array(random_imgs))

Os resultados não ficaram ideais, na medida que, apesar de os efeitos dos cílios terem sido de fato minimizados, muitos dos detalhes das íris acabaram sendo retirados.

## Borramento dos contornos da imagem

Este procedimento foi aplicado visando a diminuir o impacto dos cílios, com o intuito de minimizar o impacto destes sobre a identificação da íris pelo módulo G6.

Primeiramente, foi aplicado o filtro de Canny em uma imagem com a equalização do histograma (para melhor identificação da diferença entre cores, que evidencia os contornos). Em seguida, nas coordenadas dos contornos, foi aplicado um filtro de borramento muito intenso.


In [ ]:
random_imgs = random.sample(edges_imgs_list, 6)
result = image_grid(np.array(random_imgs))

As imagens mostram que os resultados não ficaram tão bons quanto esperado. O filtro suavizou muito pouco os contornos. Outro problema foi que alguns contornos dentro da íris foram afetados, o que poderia afetar negativamente o percentual de identificação do modelo.

# 3. Isolando o olho

## Isolamento do olho por meio da binarização da imagem

Para eliminar as partes externas ao olho que atrapalham a identificação e manter os elementos de interesse apenas, foi realizada uma binarização da imagem, com o intuito de identificar os olhos. Em seguida, as partes que não correspondiam às regiões de interesse foram eliminadas da figura, substituídas por um fundo branco. Por fim, foi aplicado um filtro bilateral suave, visando a borrar um pouco os cílios que atrapalhavam a identificação 

In [ ]:
random_imgs = random.sample(eyes2_list, 6)
result = image_grid(np.array(random_imgs))

Os resultados não ficaram tão bons quanto esperado, pois a identificação de contornos não é perfeita e algumas lacunas podem ser identificadas dentro dos olhos. Apesar disso, a maior parte dos olhos parece ter sido isolada.

## Isolamento do olho por detecção do maior contorno

Para melhorar os resultados, foi aplicado um procedimento em que é realizada uma binarização da imagem seguida de uma identificação de contornos. Ao fim, apenas aquele elemento com o maior contorno é mantido.

In [ ]:
random_imgs = random.sample(eye_imgs_list, 6)
result = image_grid(np.array(random_imgs))

Como mostram as figuras, em alguns casos o procedimento funcionou muito bem, permitindo o isolamento completo do olho do restante da imagem. Contudo, em muitos casos, os contornos não ficaram muito bem definidos, fazendo com que algumas partes externas ao olho fossem mantidas na imagem. 
Ainda assim, as falhas resultantes do procedimento anterior foram em sua maior parte eliminadas.

## Isolando a íris por detecção de círculos

Nesta etapa, foi aplicado o Hough Circles da OpenCv para detectar área correspondete à íris e destacar ela, minimizando os detalhes de outras partes da imagem.

In [ ]:
processed_isolated_eye = []
processed_isolated_eye_details = []
processed_isolated_eye_details_bright=[]

for dir in range(0,60):
      for file in range(0,3):
        src_path=f"images2/00{dir:02d}/00{dir:02d}_0{file:02d}.bmp"
        if os.path.exists(src_path):
                    
                    img_gray = cv.imread(src_path, cv.IMREAD_GRAYSCALE) 
                    
                    # pré processamento da imagem
                    img=cv.equalizeHist(img_gray)
                    img = cv2.medianBlur(img,5)
                    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    cimg_original = cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR)
                    img_final = cimg
                    img_gray = cv.add(img,127,np.median(img_gray.flatten()))
                    
                    height,width = img.shape
                    blank_image = cv.medianBlur(cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR), 11)
                    
                    # detecção de círculos
                    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=63,param2=70,minRadius=60,maxRadius=180)
                    r = 0
                    mask = np.zeros((height,width), np.uint8)
                    if circles is not None:
                        for i in circles[0,:]:
                                mask = np.zeros((height,width), np.uint8)
                                circle_img = cv2.circle(mask,(int(i[0]),int(i[1])),int(i[2]),(255,255,255),thickness=-1)
                        masked_data = cv2.bitwise_and(img_final, img_final, mask=circle_img)
                        _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
                        contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        x,y,w,h = cv2.boundingRect(np.float32(contours[0]))
                        crop = masked_data[y:y+h,x:x+w]

                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_BGR2GRAY)
                        masked_data_bright =increase_brightness(masked_data)
                        masked_data = cv2.cvtColor(masked_data,cv2.COLOR_BGR2GRAY)
                        masked_data_bright = cv2.cvtColor(masked_data_bright,cv2.COLOR_BGR2GRAY)

                        for l in range(height):
                            for c in range(width):
                              if masked_data[l,c] != 0:
                                blank_image[l,c]=masked_data[l,c]
                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_GRAY2BGR)
                        img_processed = (blank_image)
                        processed_isolated_eye_details.append(img_processed)
                                                    

In [ ]:
random_imgs = random.sample(processed_isolated_eye_details, 6)
result = image_grid(np.array(random_imgs))

Como podemos ver, a íris ficou isolada na imagem, conforme esperado. No entanto, em alguns casos, o círculo desenhado não cobre a íris inteira, o que pode atrapalhar a identificação. Além disso, a parte externa à íris ficou clara demais na imagem, de forma que mal pode ser identificada.

### Melhorando os resultados

Nas etapas a seguir, a área da íris foi destacada na própria imagem original. Primeiramente, foi aplicada uma equalização de histograma na área de interesse. Os resultados são mostrados abaixo:

In [ ]:
processed_isolated_eye = []
processed_isolated_eye_details = []
processed_isolated_eye_details_bright=[]

circles_identified_images = []
for dir in range(0,60):
      for file in range(0,3):
        src_path=f"images2/00{dir:02d}/00{dir:02d}_0{file:02d}.bmp"
        if os.path.exists(src_path):
                    img_gray = cv.imread(src_path, cv.IMREAD_GRAYSCALE) 
##### pré processamento da imagem #####
                    img=cv.equalizeHist(img_gray)
                    img = cv2.medianBlur(img,5)
                    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    cimg_original = cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR)
                    img_final = cimg
                    height,width = img.shape
                    blank_image=cimg_original
##### detecção de círculos #####
                    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=63,param2=70,minRadius=90,maxRadius=180)
                    r = 0
                    mask = np.zeros((height,width), np.uint8)
                    if circles is not None:
                        for i in circles[0,:]:
                                mask = np.zeros((height,width), np.uint8)
                                circle_img = cv2.circle(mask,(int(i[0]),int(i[1])),int(i[2]),(255,255,255),thickness=-1)

##### deixando a íris em evidência #####
                        masked_data = cv2.bitwise_and(img_final, img_final, mask=circle_img)
                        _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
                        contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        x,y,w,h = cv2.boundingRect(np.float32(contours[0]))
                        crop = masked_data[y:y+h,x:x+w]
                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_BGR2GRAY)
                        masked_data = cv2.cvtColor(masked_data,cv2.COLOR_BGR2GRAY)
                        hist_hsv = hsv_histeq(img_gray)
                        for l in range(height):
                            for c in range(width):
                              if masked_data[l,c] != 0:
                                blank_image[l,c]=hist_hsv[l,c]
                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_GRAY2BGR)
                        circles_identified_images.append(blank_image)
                    

In [ ]:
random_imgs = random.sample(circles_identified_images, 6)
result = image_grid(np.array(random_imgs))

As imagens acima mostraram resultados interessantes. O procedimento não funcionou muito bem para todas as imagens, porém, para aquelas em que a íris foi identificada corretamente, a região de interesse obteve maior destaque. 

O mesmo procedimento acima foi aplicado agora com a equalização de histograma adaptativa:

In [ ]:
processed_isolated_eye = []
processed_isolated_eye_details = []
processed_isolated_eye_details_bright=[]

circles_identified_images = []
for dir in range(0,60):
      for file in range(0,3):
        src_path=f"images2/00{dir:02d}/00{dir:02d}_0{file:02d}.bmp"
        if os.path.exists(src_path):
                    img_gray = cv.imread(src_path, cv.IMREAD_GRAYSCALE) 
##### pré processamento da imagem #####
                    img=cv.equalizeHist(img_gray)
                    img = cv2.medianBlur(img,5)
                    cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                    cimg_original = cv2.cvtColor(img_gray,cv2.COLOR_GRAY2BGR)
                    img_final = cimg
                    height,width = img.shape
                    blank_image=cimg_original
                    
##### detecção de círculos #####
                    circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=63,param2=70,minRadius=90,maxRadius=180)
                    r = 0
                    mask = np.zeros((height,width), np.uint8)
                    if circles is not None:
                        for i in circles[0,:]:
                                mask = np.zeros((height,width), np.uint8)
                                circle_img = cv2.circle(mask,(int(i[0]),int(i[1])),int(i[2]),(255,255,255),thickness=-1)

##### deixando a íris em evidência #####
                        masked_data = cv2.bitwise_and(img_final, img_final, mask=circle_img)
                        _,thresh = cv2.threshold(mask,1,255,cv2.THRESH_BINARY)
                        contours,_ = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
                        x,y,w,h = cv2.boundingRect(np.float32(contours[0]))
                        crop = masked_data[y:y+h,x:x+w]
                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_BGR2GRAY)
                        masked_data = cv2.cvtColor(masked_data,cv2.COLOR_BGR2GRAY)
                        clahe = apply_clahe(img_gray)
                        for l in range(height):
                            for c in range(width):
                              if masked_data[l,c] != 0:
                                blank_image[l,c]=clahe[l,c]
                        blank_image = cv2.cvtColor(blank_image,cv2.COLOR_GRAY2BGR)
                        circles_identified_images.append(blank_image)
                    

In [ ]:
random_imgs = random.sample(circles_identified_images, 6)
result = image_grid(np.array(random_imgs))

Este filtro realçou ainda mais os detalhes da íris, o que tende a facilitar a identificação.

# Isolando e aumentando a íris

O código a seguir detecta os círculos correspondentes à íris e recorta o restante da imagem, de forma a dar zoom  nas partes de interesse.

In [ ]:
processed_isolated_eye = []
processed_isolated_eye_sharp = []
processed_isolated_eye_bright = []
processed_isolated_eye_sharp_bright = []
clahe_eye=[]
for dir in range(0,60):
      for file in range(0,3):
        src_path=f"images2/00{dir:02d}/00{dir:02d}_0{file:02d}.bmp"
        if os.path.exists(src_path):
                
                if os.path.exists(src_path):
                    img_gray = cv.imread(src_path, cv.IMREAD_GRAYSCALE) 
                img=cv.equalizeHist(img_gray)
                img = cv2.medianBlur(img,5)
                cimg = cv2.cvtColor(img,cv2.COLOR_GRAY2BGR)
                img_final = cimg
                circles = cv2.HoughCircles(img,cv2.HOUGH_GRADIENT,1,10,param1=63,param2=70,minRadius=60,maxRadius=180)
                height,width = img.shape
                blank_image = np.zeros((height,width,3), np.uint8)
                r = 0
                mask = np.zeros((height,width), np.uint8)
                if circles is not None:
                    for i in circles[0,:]:
                          h1=(int(i[1])-int(i[2]))-50
                          h2=int(i[1])+int(i[2])+50
                          w1=(int(i[0])-int(i[2]))-50
                          w2=int(i[0])+int(i[2])+50

                          height = abs(h2-h1)
                          width=abs(w2-w1)
                          blank_image = np.zeros((height,width,3), np.uint8)

                          blank_image = img_final[int(h1):int(h2),int(w1):int(w2)]
                          hb,wb,cb = blank_image.shape
                          h,w = img_gray.shape
                          if h>1 and w >1 and hb>1 and cb>1 and blank_image is not None:
                              try:
                                    quadro = np.zeros((h,w,3), np.uint8)
                                    quadro[:]=255
                                    ratio_height = h/hb
                                    ratio_width = w/wb
                                    ratio = min(ratio_width,ratio_height)
                                    w_new=int(ratio*wb)
                                    h_new=int(ratio*hb)
                                    resized = cv.resize(blank_image, (w_new,h_new))
                                    quadro=resized
                                    print(image.shape)
                              except:
                                    pass
                    kernel = np.array([[0, -1, 0],
                                       [-1, 5,-1],
                                      [0, -1, 0]])
                    if hb>0 and cb>0:
                            image_sharp = cv2.filter2D(src=quadro, ddepth=-1, kernel=kernel)
                            cimg = cv2.cvtColor(image_sharp,cv2.COLOR_BGR2GRAY)
                            processed_isolated_eye_sharp.append(cimg)
                            img_processed=quadro
                            image_sharp=increase_brightness(img_processed)
                            cimg = cv2.cvtColor(img_processed,cv2.COLOR_BGR2GRAY)
                            processed_isolated_eye_sharp_bright.append(cimg)

                            img_processed = cv.detailEnhance(quadro)
                            cimg = cv2.cvtColor(img_processed,cv2.COLOR_BGR2GRAY)
                            processed_isolated_eye.append(cimg)
                            
                            img_processed=increase_brightness(img_processed)
                            cimg = cv2.cvtColor(img_processed,cv2.COLOR_BGR2GRAY)
                            processed_isolated_eye_bright.append(cimg)
                            cimg = cv2.cvtColor(quadro,cv2.COLOR_BGR2GRAY)
                            clahe=apply_clahe(cimg)
                            clahe_eye.append(clahe)

In [ ]:
random_imgs = random.sample(processed_isolated_eye_sharp, 6)
result = image_grid(np.array(random_imgs))

In [ ]:
random_imgs = random.sample(processed_isolated_eye_sharp, 6)
result = image_grid(np.array(random_imgs))

In [ ]:
random_imgs = random.sample(processed_isolated_eye_sharp_bright, 6)
result = image_grid(np.array(random_imgs))

In [ ]:
random_imgs = random.sample(processed_isolated_eye, 6)
result = image_grid(np.array(random_imgs))

O procedimento de dar zoom na íris não foi benéfico para facilitar a identificação da imagem. Isso porque em muitas imagens, partes importantes para a idenficação foram cortadas. Além disso, ao aumentar o tamanho da íris, os parâmetros do modelo para a identificação dos círculos não se adequam mais. Isso piora muito a identificação.

# Escolha dos melhores procedimentos

Com base nos resultados apresentados acima, os filtros que apresentaram os melhores resultados foram:
* Equalização do histograma no canal V
* Borramento mais suave (gaussiano)
* Equalização de histograma adaptativa (CLAHE) - com o coeficiente mais suave e o mais intenso
* Aplicação do filtro de convolução 
* Intensificação de detalhes pela função detailEnhance
* Destaque da íris na imagem com equalização de histograma adaptativa

Estes procedimentos serão aplicados sequencialmente para a identificação do modelo no Notebook da parte 02.